# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 8:51AM,254522,12,HH-39258,Hush Hush,Released
1,Jan 9 2023 8:46AM,254519,22,638025,"NBTY Global, Inc.",Released
2,Jan 9 2023 8:46AM,254518,22,638024,"NBTY Global, Inc.",Released
3,Jan 9 2023 8:44AM,254517,20,9009424,"Granules Pharmaceuticals, Inc.",Released
4,Jan 9 2023 8:43AM,254516,10,0086284898,ISDIN Corporation,Released
5,Jan 9 2023 8:43AM,254516,10,0086284904,ISDIN Corporation,Released
6,Jan 9 2023 8:43AM,254516,10,0086284905,ISDIN Corporation,Released
7,Jan 9 2023 8:43AM,254516,10,0086284913,ISDIN Corporation,Released
8,Jan 9 2023 8:43AM,254516,10,0086284944,ISDIN Corporation,Released
9,Jan 9 2023 8:43AM,254516,10,0086285010,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,254516,Released,9
7,254517,Released,1
8,254518,Released,1
9,254519,Released,1
10,254522,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
254516,NaN,9.0
254517,NaN,1.0
254518,NaN,1.0
254519,NaN,1.0
254522,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
254504,0.0,2.0
254505,2.0,9.0
254511,0.0,8.0
254512,0.0,11.0
254515,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
254504,0,2
254505,2,9
254511,0,8
254512,0,11
254515,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,254504,0,2
1,254505,2,9
2,254511,0,8
3,254512,0,11
4,254515,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,254504,,2
1,254505,2,9
2,254511,,8
3,254512,,11
4,254515,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 8:51AM,254522,12,Hush Hush
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc."
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc."
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc."
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation
13,Jan 9 2023 8:30AM,254504,10,Emerginnova
15,Jan 9 2023 8:30AM,254505,10,ISDIN Corporation
26,Jan 9 2023 8:02AM,254515,20,"Exact-Rx, Inc."
27,Jan 9 2023 7:46AM,254512,19,"Innogenix, LLC"
38,Jan 9 2023 7:40AM,254511,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Jan 9 2023 8:51AM,254522,12,Hush Hush,,1
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc.",,1
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc.",,1
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc.",,1
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation,,9
5,Jan 9 2023 8:30AM,254504,10,Emerginnova,,2
6,Jan 9 2023 8:30AM,254505,10,ISDIN Corporation,2,9
7,Jan 9 2023 8:02AM,254515,20,"Exact-Rx, Inc.",,1
8,Jan 9 2023 7:46AM,254512,19,"Innogenix, LLC",,11
9,Jan 9 2023 7:40AM,254511,19,"AdvaGen Pharma, Ltd",,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 8:51AM,254522,12,Hush Hush,1,
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc.",1,
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc.",1,
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc.",1,
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation,9,
5,Jan 9 2023 8:30AM,254504,10,Emerginnova,2,
6,Jan 9 2023 8:30AM,254505,10,ISDIN Corporation,9,2
7,Jan 9 2023 8:02AM,254515,20,"Exact-Rx, Inc.",1,
8,Jan 9 2023 7:46AM,254512,19,"Innogenix, LLC",11,
9,Jan 9 2023 7:40AM,254511,19,"AdvaGen Pharma, Ltd",8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 8:51AM,254522,12,Hush Hush,1,
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc.",1,
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc.",1,
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc.",1,
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 8:51AM,254522,12,Hush Hush,1,NaN
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc.",1,NaN
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc.",1,NaN
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc.",1,NaN
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation,9,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Jan 9 2023 8:51AM,254522,12,Hush Hush,1,0.0
1,Jan 9 2023 8:46AM,254519,22,"NBTY Global, Inc.",1,0.0
2,Jan 9 2023 8:46AM,254518,22,"NBTY Global, Inc.",1,0.0
3,Jan 9 2023 8:44AM,254517,20,"Granules Pharmaceuticals, Inc.",1,0.0
4,Jan 9 2023 8:43AM,254516,10,ISDIN Corporation,9,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
10,763525,20,2.0
12,254522,1,0.0
19,509023,19,0.0
20,509032,2,0.0
22,509037,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,10,763525,20,2.0
1,12,254522,1,0.0
2,19,509023,19,0.0
3,20,509032,2,0.0
4,22,509037,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,10,20,2.0
1,12,1,0.0
2,19,19,0.0
3,20,2,0.0
4,22,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,12,Released,1.0
2,19,Released,19.0
3,20,Released,2.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Released,20.0,1.0,19.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Released,20.0,1.0,19.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Released,20.0,1.0,19.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()